# Packages


In [42]:
import json
import argparse
import sys,os
import random
import datetime
import pandas as pd
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import csv
# pySODM packages
from pySODM.models.base import ODEModel
from pySODM.optimization import pso, nelder_mead
from pySODM.optimization.utils import add_poisson_noise, assign_theta, variance_analysis
from pySODM.optimization.mcmc import perturbate_theta, run_EnsembleSampler, emcee_sampler_to_dictionary
from pySODM.optimization.objective_functions import log_posterior_probability, log_prior_uniform, ll_gaussian
# COVID-19 package
from covid19_DTM.data.sciensano import get_sciensano_COVID19_data

# data and samples


In [43]:
abs_dir = os.getcwd()

In [44]:
rel_dir = '../../data/PHM/interim/UZG'

# Postponed healthcare data
file_name = '2020_2021_normalized.csv'
types_dict = {'APR_MDC_key': str}
# mean data
hospitalizations_normalized = pd.read_csv(os.path.join(abs_dir,rel_dir,file_name),index_col=[0,1],dtype=types_dict,parse_dates=True)['mean']
hospitalizations_normalized = hospitalizations_normalized.reorder_levels(['date','APR_MDC_key'])
hospitalizations_normalized=hospitalizations_normalized.sort_index()
hospitalizations_normalized=hospitalizations_normalized.reindex(hospitalizations_normalized.index.rename('MDC',level=1))
MDC_keys = hospitalizations_normalized.index.get_level_values('MDC').unique().values

# lower and upper quantiles
hospitalizations_normalized_quantiles = pd.read_csv(os.path.join(abs_dir,rel_dir,file_name),index_col=[0,1],dtype=types_dict,parse_dates=True).loc[(slice(None), slice(None)), ('q0.025','q0.975')]
hospitalizations_normalized_quantiles = hospitalizations_normalized_quantiles.reorder_levels(['date','APR_MDC_key'])
hospitalizations_normalized_quantiles=hospitalizations_normalized_quantiles.sort_index()
hospitalizations_normalized_quantiles=hospitalizations_normalized_quantiles.reindex(hospitalizations_normalized_quantiles.index.rename('MDC',level=1))

file_name = 'MZG_2016_2021.csv'
types_dict = {'APR_MDC_key': str}
hospitalizations = pd.read_csv(os.path.join(abs_dir,rel_dir,file_name),index_col=[0,1,2,3],dtype=types_dict).squeeze()
hospitalizations = hospitalizations.groupby(['APR_MDC_key','date']).sum()
hospitalizations.index = hospitalizations.index.set_levels([hospitalizations.index.levels[0], pd.to_datetime(hospitalizations.index.levels[1])])
hospitalizations = hospitalizations.reorder_levels(['date','APR_MDC_key'])
hospitalizations=hospitalizations.sort_index()
hospitalizations=hospitalizations.reindex(hospitalizations.index.rename('MDC',level=1))

# COVID-19 data
covid_data, _ , _ , _ = get_sciensano_COVID19_data(update=False)
new_index = pd.MultiIndex.from_product([pd.to_datetime(hospitalizations_normalized.index.get_level_values('date').unique()),covid_data.index.get_level_values('NIS').unique()])
covid_data = covid_data.reindex(new_index,fill_value=0)
df_covid_H_in = covid_data['H_in'].loc[:,40000]
df_covid_H_tot = covid_data['H_tot'].loc[:,40000]
df_covid_dH = df_covid_H_tot.diff().fillna(0)

# hospitalisation baseline
file_name = 'MZG_baseline.csv'
types_dict = {'APR_MDC_key': str, 'week_number': int, 'day_number':int}
hospitalization_baseline = pd.read_csv(os.path.join(abs_dir,rel_dir,file_name),index_col=[0,1,2,3],dtype=types_dict).squeeze()
hospitalization_baseline = hospitalization_baseline.groupby(['APR_MDC_key','week_number','day_number']).mean()
hospitalization_baseline = hospitalization_baseline.ewm(14).mean()

# mean hospitalisation length
file_name = 'MZG_residence_times.csv'
types_dict = {'APR_MDC_key': str, 'age_group': str, 'stay_type':str}
residence_times = pd.read_csv(os.path.join(abs_dir,rel_dir,file_name),index_col=[0,1,2],dtype=types_dict).squeeze()
mean_residence_times = residence_times.groupby(by=['APR_MDC_key']).mean()
mean_residence_times['covid']=10

In [77]:
rel_dir = '../../data/PHM/interim/model_parameters'

# Load samples
file_name = 'PI_SAMPLES.json'
f = open(os.path.join(abs_dir,rel_dir,file_name))
samples_dict_PI = json.load(f)

file_name = 'queuing_SAMPLES.json'
f = open(os.path.join(abs_dir,rel_dir,file_name))
samples_dict_queuing = json.load(f)

file_name = 'queuing_SETTINGS.json'
f = open(os.path.join(abs_dir,rel_dir,file_name))
settings_queuing = json.load(f)

# models and functions


## models

In [63]:
class Constrained_PI_Model(ODEModel):
    """
    Test constrainted PI model
    """
    
    state_names = ['H','E']
    parameter_names = ['covid_H','covid_dH']
    parameter_stratified_names = ['Kp', 'Ki', 'alpha','gamma', 'covid_capacity']
    dimension_names = ['MDC']
    
    @staticmethod
    def integrate(t, H, E, covid_H, covid_dH, Kp, Ki, alpha, gamma, covid_capacity):

        epsilon = 1-H
        dE = epsilon - gamma*E
        u = np.where(covid_H <= covid_capacity, Kp * np.where(E<=0,epsilon,0) + Ki * np.where(E>0,E,0), 0)
        dH = -alpha*covid_dH + u

        return dH, dE 

In [64]:
class PI_Model(ODEModel):
    """
    Test PI model
    """
    
    state_names = ['H','E']
    parameter_names = ['covid_H']
    parameter_stratified_names = ['Kp', 'Ki', 'alpha', 'gamma']
    dimension_names = ['MDC']
    
    @staticmethod
    def integrate(t, H, E, covid_H, Kp, Ki, alpha, gamma):

        epsilon = 1-H
        dE = epsilon - gamma*E
        u = Kp*epsilon + Ki*E
        dH = -alpha*covid_H + u

        return dH, dE 

In [65]:
class Queuing_Model(ODEModel):
    """
    Test model for postponed health_care using a waiting queue before hospitalization
    """
    
    state_names = ['W','H','R','NR','X']
    parameter_names = ['X_tot','f_UZG']
    parameter_stratified_names = ['A','gamma','epsilon','sigma']
    dimension_names = ['MDC']
    
    @staticmethod
    def integrate(t, W, H, R, NR, X, A, X_tot, f_UZG, gamma, epsilon, sigma):
        X_new = (X_tot-sum(H - (1/gamma*H))) * (sigma*(A+W))/sum(sigma*(A+W))

        W_to_H = np.where(W>X_new,X_new,W)
        W_to_NR = epsilon*(W-W_to_H)
        A_to_H = np.where(A>(X_new-W_to_H),(X_new-W_to_H),A)
        A_to_W = A-A_to_H
        
        dX = -X + X_new - W_to_H - A_to_H
        dW = A_to_W - W_to_H - W_to_NR
        dH = A_to_H + W_to_H - (1/gamma*H)
        dR = (1/gamma*H)
        dNR = W_to_NR
        
        return dW, dH, dR, dNR, dX

## TDF

In [49]:
from functools import lru_cache

class get_covid_H():

    def __init__(self, data):
        self.data = data

    def H_wrapper_func(self, t, states, param):
        return self.__call__(t)

    @lru_cache()
    def __call__(self, t):
        t = pd.to_datetime(t).round(freq='D')
        try:
            covid_H = self.data.loc[t]
        except:
            covid_H = 0
        return covid_H  

In [50]:
class get_covid_dH():

    def __init__(self, data):
        self.data = data

    def dH_wrapper_func(self, t, states, param):
        return self.__call__(t)

    @lru_cache
    def __call__(self, t):
        t = pd.to_datetime(t).round(freq='D')
        try:
            covid_dH = self.data.loc[t]
        except:
            covid_dH = 0

        return covid_dH

In [51]:
class get_A():
    def __init__(self, baseline,mean_residence_times,df_covid_H_in):
        self.baseline = baseline
        self.mean_residence_times = mean_residence_times
        self.df_covid_H_in = df_covid_H_in
    
    def A_wrapper_func(self, t, states, param, MDC,f_UZG):
        return self.__call__(t,MDC,f_UZG)
    
    #@lru_cache()
    def __call__(self, t, MDC,f_UZG):
        covid_idx = int(np.where(MDC=='covid')[0])
        MDC_without_covid = np.delete(MDC,covid_idx)

        t_string = t.strftime('%Y-%m-%d')
        week = t.isocalendar().week
        day = t.isocalendar().weekday

        A = (self.baseline.loc[(MDC_without_covid,week,day)]/self.mean_residence_times.loc[MDC_without_covid]).values
        covid_H_in = self.df_covid_H_in.loc[t_string]*f_UZG
        A = np.insert(A,covid_idx,covid_H_in)

        return A 

## init models

'2020-08-01'

In [69]:
# PI model
PI_samples_per_MDC = samples_dict_PI['PI']
MDC = list(PI_samples_per_MDC.keys())
pars = PI_samples_per_MDC[MDC[0]]['parameters']

param_dict = {}
for param in pars:
    param_array = np.array([])
    for disease in MDC:
        param_array = np.append(param_array,PI_samples_per_MDC[disease][param][0][0])
    param_dict.update({param:param_array})

param_dict.update({'covid_H':0})
H_function = get_covid_H(df_covid_H_tot).H_wrapper_func
time_dependent_functions = {'covid_H':H_function}

init_states = {'H':np.ones(len(MDC))}
coordinates={'MDC': MDC}

PI_model = PI_Model(init_states,param_dict,coordinates,time_dependent_functions)

# constrained PI model
PI_constrained_samples_per_MDC = samples_dict_PI['constrained_PI']
MDC = list(PI_constrained_samples_per_MDC.keys())
pars = PI_constrained_samples_per_MDC[MDC[0]]['parameters']

param_dict = {}
for param in pars:
    param_array = np.array([])
    for disease in MDC:
        param_array = np.append(param_array,PI_constrained_samples_per_MDC[disease][param][0][0])
    param_dict.update({param:param_array})

param_dict.update({'covid_dH':0,'covid_H':0})
dH_function = get_covid_dH(df_covid_dH).dH_wrapper_func
H_function = get_covid_H(df_covid_H_tot).H_wrapper_func
time_dependent_functions = {'covid_dH':dH_function,'covid_H':H_function}

init_states = {'H':np.ones(len(MDC))}
coordinates={'MDC': MDC}

constrained_PI_model = Constrained_PI_Model(init_states,param_dict,coordinates,time_dependent_functions)

# Queuing model
pars = samples_dict_queuing['parameters']

param_dict = {}
for param in pars:
    if param == 'f_UZG':
        param_dict.update({param:samples_dict_queuing['f_UZG'][0]})
    else:
        param_array = np.array([])
        for n, disease in enumerate(MDC):
            param_array = np.append(param_array,samples_dict_queuing[param][n][0])
        param_dict.update({param:param_array})
param_dict.update({'X_tot':1049})

start_date = settings_queuing['start_calibration']
covid_H_in = df_covid_H_in.loc[start_date]*f_UZG
covid_H = df_covid_H_tot.loc[t_string]*f_UZG
H_init = hospitalization_baseline.loc[(MDC_sim,start_date.isocalendar().week,start_date.isocalendar().weekday)].values
H_init = np.insert(H_init,np.where(MDC_sim=='covid')[0],covid_H)
A = H_init/mean_residence_times.loc[MDC_sim]

param_dict.update({'A':0,'covid_H':0})
dH_function = get_covid_dH(df_covid_dH).dH_wrapper_func
H_function = get_covid_H(df_covid_H_tot).H_wrapper_func
time_dependent_functions = {'covid_dH':dH_function,'covid_H':H_function}

init_states = {'H':np.ones(len(MDC))}
coordinates={'MDC': MDC}

constrained_PI_model = Constrained_PI_Model(init_states,param_dict,coordinates,time_dependent_functions)

In [85]:
len(samples_dict_queuing['sigma'])

29

In [76]:
samples_dict_queuing['parameters']

['f_UZG', 'epsilon', 'sigma']

In [60]:
PI_samples_per_MDC[disease][param][0][0]

7.53417755512082e-05

In [ ]:
model = model_class(init_states,params_dict,coordinates,time_dependent_parameters=time_dependent_parameters)